In [ ]:
# Active Learning - Uncertainty Sampling

import numpy as np
import pandas as pd
import os
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.calibration import CalibratedClassifierCV
from warnings import simplefilter
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.linear_model import LogisticRegression

from google.colab import drive


simplefilter(action='ignore', category=FutureWarning)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported fro

In [ ]:
# Συνδέομαι στο google drive για να μπορώ να αποθηκεύω και να ανακτώ μοντέλα από εκεί
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Καθορίζω την διαδρομή στο google drive και φορτώνω στην μνήμη το σύνολο των δεδομένων
path = 'drive/My Drive/bank_credit'
dt = pd.read_csv(os.path.join(path, 'creditcard.csv'))

In [ ]:
# Επισκόπηση λίγων δεδομένων των δύο κλάσεων
a = dt.loc[dt['Class']==0].sample(3)
b = dt.loc[dt['Class']==1].sample(5)
c = pd.concat([a,b])
c

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
113019,72896.0,-1.675121,0.615868,1.401999,0.278928,0.719191,-1.331490,0.384899,0.257686,-1.047847,-0.989240,-0.513598,0.838772,1.400674,0.325724,0.216494,0.228599,-0.512950,-0.165069,-0.824478,0.110574,0.214499,0.198891,-0.384372,0.434770,0.750188,-0.414120,-0.068170,-0.132026,6.16,0
9574,14383.0,1.091960,0.740444,0.199043,2.668172,0.521627,-0.276681,0.542297,-0.303731,0.043376,0.264391,0.835861,-1.919303,2.760137,1.761523,-0.400927,-0.040658,0.468439,-0.809593,-1.472637,-0.062268,-0.093770,-0.066425,-0.121592,0.059114,0.655079,0.049942,-0.051991,0.011691,53.89,0
54570,46509.0,-1.555932,1.386686,0.416229,-1.401255,-0.827749,-0.759278,-0.377315,1.044272,0.033310,-0.297558,0.564177,0.499119,-0.683169,0.743402,-0.118389,1.184654,-0.699242,0.369488,-0.411928,0.014507,-0.062894,-0.340920,0.057258,0.014598,-0.116899,0.742058,0.134003,0.035662,3.84,0
213116,139117.0,-3.975939,-1.244939,-3.707414,4.544772,4.050676,-3.407679,-5.063118,1.007042,-3.190158,-4.250717,4.182162,-4.563675,1.182503,-6.964972,1.115340,-4.997332,-6.419539,-1.183593,3.569733,2.109403,1.059737,-0.037395,0.348707,-0.162929,0.410531,-0.123612,0.877424,0.667568,8.30,1
220725,142280.0,-1.169203,1.863414,-2.515135,5.463681,-0.297971,1.364918,0.759219,-0.118861,-2.293921,-0.423784,2.375876,-3.244827,-0.556619,-5.152475,0.050906,-1.022045,-1.646505,0.126460,1.819013,-0.299374,-0.393090,-0.708692,0.471309,-0.078616,-0.544655,0.014777,-0.240930,-0.781055,324.59,1
102442,68207.0,-13.192671,12.785971,-9.906650,3.320337,-4.801176,5.760059,-18.750889,-37.353443,-0.391540,-5.052502,4.406806,-4.610756,-1.909488,-9.072711,-0.226074,-6.211557,-6.248145,-3.149247,0.051576,-3.493050,27.202839,-8.887017,5.303607,-0.639435,0.263203,-0.108877,1.269566,0.939407,1.00,1
191544,129308.0,0.054682,1.856500,-4.075451,4.100098,-0.800931,-0.292502,-2.317431,1.189747,-0.786238,-3.906746,2.993489,-3.613125,-1.727297,-7.364639,-0.343296,-2.577635,-0.515302,1.109107,2.351151,0.509559,0.618248,0.800932,0.130016,0.288946,-0.366658,0.030307,0.431182,0.110698,80.90,1
152019,96717.0,-3.705856,4.107873,-3.803656,1.710314,-3.582466,1.469729,-9.621560,-11.913105,-0.322297,-6.625692,2.175230,-4.381078,2.063299,-0.673818,1.400763,-4.254927,-5.160213,-1.302526,2.594453,3.639603,-5.498772,2.941475,0.916236,-0.255504,-0.183835,-0.584539,-0.315484,-0.097223,1.00,1


In [ ]:
# Δημιουργώ ένα χαρακτηριστικό πυκνότητας συναλλαγών (Density) και ένα πραγματικής ώρας (Actual Time)
# θεωρώντας ότι μπορεί να υπάρχει κάποια συσχέτιση με τις κακόβουλες κινήσεις
# Ενώ παράλληλα διαγράφω το χαρακτηριστικό Time
period = 30 * 60 # secs
half_period = int(period / 2)
a = np.histogram(dt['Time'], bins=np.linspace(0, 172800, 172801))[0]

density48h = [np.mean(a[[j % 172800 for j in range(-half_period, +half_period)]])]
for i in range(1, len(a)):
  density48h.append(density48h[-1] + (a[(i + half_period - 1) % 172800] - a[(i - half_period) % 172800]) / period)

density48h = np.array(density48h)
density48h = density48h / density48h.max()

dt['Density'] = density48h[dt['Time'].astype(int).values]
dt['Density'].reset_index(drop=True)

dt['Actual_Time_cos'] = dt['Time'].apply(lambda x: np.cos(np.mod(x, 86400) / 86400 * 2 * np.pi))
dt['Actual_Time_sin'] = dt['Time'].apply(lambda x: np.sin(np.mod(x, 86400) / 86400 * 2 * np.pi))

dt.drop('Time', inplace=True, axis='columns')

In [ ]:
# Μετατρέπω το χαρακτηριστικό Amount με λογαριθμική κανονικοποίηση
amounts = dt['Amount'].copy()
dt['Amount'] = np.log(dt['Amount']+2)

In [ ]:
# Χωρίζω το σύνολο δεδομένων σε εκπαίδευσης, επικύρωσης και ελέγχου
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
seed = 0

x_tr, x_te, y_tr, y_te = train_test_split(dt.loc[:, dt.columns != 'Class'], dt['Class'], test_size=.3, random_state=seed)

va_size = int(len(x_tr) * .2)
x_tr, x_UN = x_tr[:va_size], x_tr[va_size:]
y_tr, y_UN = y_tr[:va_size], y_tr[va_size:]

print(x_tr.shape, x_UN.shape, x_te.shape)

(39872, 32) (159492, 32) (85443, 32)


In [ ]:
# Δημιουργία ενός αριθμού από υποσύνολα δεδομένων τα οποία έχουν ίσο αριθμό παραδειγμάτων των δύο κλάσεων.
# Με αυτά εκπαιδεύονται μοντέλα Random Forest και η έκδσοση απόφασης γίνεται με ψηφοφορία.
# Το bagging μοντέλο εκπαιδεύεται και αποθηκεύεται, ώστε να μην χρειάζεται σε κάθε εκτέλεση του notebook

y0 = y_tr[y_tr==0]
x0 = x_tr[y_tr==0]

y1 = y_tr[y_tr==1]
x1 = x_tr[y_tr==1]

ln = len(y1)
number_of_subsets = 200

print(f'iso {number_of_subsets}...')
mod_iso = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=200), n_estimators=number_of_subsets)
model_iso = CalibratedClassifierCV(base_estimator=mod_iso, cv=3, method='isotonic')
model_iso.fit(x_tr, y_tr)

joblib.dump(model_iso, path + '/iso_bal_bag_rf_200_gini_200_active_learning_cl.joblib')

iso 200...


['drive/My Drive/bank_credit/iso_bal_bag_rf_200_gini_200_active_learning_cl.joblib']

In [ ]:
# Ανάκληση του εκπαιδευμένου μοντέλου και υπολογισμός προβλέψεων στο σύνολο ελέγχου

model_iso = joblib.load(path + '/iso_bal_bag_rf_200_gini_200_active_learning_cl.joblib')

y_preds = model_iso.predict_proba(x_te.to_numpy())

y_preds = y_preds.argmax(axis=1)

print(confusion_matrix(y_te, y_preds).T)
print()
print(classification_report(y_te, y_preds))


[[85278    33]
 [   18   114]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.86      0.78      0.82       147

    accuracy                           1.00     85443
   macro avg       0.93      0.89      0.91     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
# Πρόβλεψη των καλιμπραρισμένων πιθανοτήτων για όλα τα μη ετικετοποιημένα δεδομένα και επιλογή των 50000 πιο αβέβαιων

# cost = 5000
# cost_per_example = 0.1

# preds = model_iso.predict_proba(x_UN.to_numpy())

# idx = np.argsort(np.abs(preds[:,0].flatten() - 0.5))[:int(cost/cost_per_example)]
 

In [ ]:
# Εκπαίδευση του μοντέλου με τα νέα επαυξημένα δεδομένα και αποθήκευσή του ώστε να μην επανεκπαιδεύεται σε κάθε νέα εκκίνηση του notebook

# x_ = np.vstack([x_UN.to_numpy()[idx], x_tr])
# y_ = y_UN.to_numpy()[idx].tolist() + y_tr.to_list()

# mod_iso_ = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=200), n_estimators=number_of_subsets)
# model_iso_ = CalibratedClassifierCV(base_estimator=mod_iso_, cv=3, method='isotonic')
# model_iso_.fit(x_, y_)

# joblib.dump(model_iso_, path + '/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty.joblib')


In [ ]:
# Ανάκληση του αποθηκευμένου μοντέλου και πρόβλεψη των κατηγοριών των παραδειγμάτων ελέγχου
# Αυτή η προσπάθεια δεν δούλεψε καλά. Έριξε την ακρίβεια του μοντέλου για τα αρνητικά παραδείγματα η επέκταση των δεδομένων με μία μόνο εκτίμηση

model_iso_ = joblib.load(path + '/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty.joblib')

y_preds_ = model_iso_.predict_proba(x_te.to_numpy())

y_preds_ = y_preds_.argmax(axis=1)

print(confusion_matrix(y_te, y_preds_).T)
print()
print(classification_report(y_te, y_preds_))

[[85283    44]
 [   13   103]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.89      0.70      0.78       147

    accuracy                           1.00     85443
   macro avg       0.94      0.85      0.89     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
# Σταδιακή επέκταση των δεδομένων εκπαίδευσης σε 10 βήματα (ανά 5000 δείγματα) και αποθήκευση
# των επιμέρους εκπαιδευμένων μοντέλων για να μην επανεκπαιδεύονται σε κάθε επανεκκίνηση του notebook

# cost = 5000
# cost_per_example = 0.1


# sps = np.linspace(0, int(cost/cost_per_example), 11)
# x_rest = x_UN.to_numpy()
# y_rest = y_UN.to_numpy()
# x_ = x_tr.to_numpy()
# y_ = y_tr.to_numpy()

# for i in range(10):
#   print(f'{i+1}/10...')
#   sp = int(sps[i+1] - sps[i])

#   preds = model_iso.predict_proba(x_rest)

#   idx = np.argsort(np.abs(preds[:,0].flatten() - 0.5))[:sp]

#   x_ = np.vstack([x_rest[idx], x_])
#   y_ = np.array(y_rest[idx].tolist() + y_.tolist())

#   mod_iso_ = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=200), n_estimators=number_of_subsets)
#   model_iso_ = CalibratedClassifierCV(base_estimator=mod_iso_, cv=3, method='isotonic')
#   model_iso_.fit(x_, y_)

#     joblib.dump(model_iso_, path + f'/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty_{i}.joblib')

#   x_rest = np.delete(x_rest, idx, axis=0)
#   y_rest = np.delete(y_rest, idx, axis=0)

In [ ]:
# Ανάκληση των αποθηκευμένων μοντέλων με τα σταδιακά αυξανόμενα δεδομένα και εκτύπωση στατιστικών

for i in range(10):
  model_iso_ = joblib.load(path + f'/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty_{i}.joblib')

  y_preds_ = model_iso_.predict_proba(x_te.to_numpy())

  y_preds_ = y_preds_.argmax(axis=1)

  print(f'Μετά από πρόσθεση {i+1}x5000 παραδειγμάτων...')
  print(confusion_matrix(y_te, y_preds_).T)
  print()
  print(classification_report(y_te, y_preds_))
  print(50*'-')

Μετά από πρόσθεση 1x5000 παραδειγμάτων...
[[85279    32]
 [   17   115]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.87      0.78      0.82       147

    accuracy                           1.00     85443
   macro avg       0.94      0.89      0.91     85443
weighted avg       1.00      1.00      1.00     85443

--------------------------------------------------
Μετά από πρόσθεση 2x5000 παραδειγμάτων...
[[85279    34]
 [   17   113]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.87      0.77      0.82       147

    accuracy                           1.00     85443
   macro avg       0.93      0.88      0.91     85443
weighted avg       1.00      1.00      1.00     85443

--------------------------------------------------
Μετά από πρόσθεση 3x5000 παραδειγμάτων...
[[85278    34]
 [   18   113]]

              precisio

In [ ]:
# Σταδιακή επέκταση των δεδομένων εκπαίδευσης σε 10 βήματα (ανά 5000 δείγματα) και αποθήκευση
# των επιμέρους εκπαιδευμένων μοντέλων για να μην επανεκπαιδεύονται σε κάθε επανεκκίνηση του notebook

cost = 5000
cost_per_example = 0.1


sps = np.linspace(0, int(cost/cost_per_example), 11)
x_rest = x_UN.to_numpy()
y_rest = y_UN.to_numpy()
x_ = x_tr.to_numpy()
y_ = y_tr.to_numpy()

for i in range(10):
  print(f'{i+1}/10...')
  sp = int(sps[i+1] - sps[i])

  preds = model_iso.predict_proba(x_rest)

  idx = np.argsort(np.abs(preds[:,0].flatten() - 0.5))[:4 * sp]
  np.random.shuffle(idx)
  idx = idx[:sp]

  x_ = np.vstack([x_rest[idx], x_])
  y_ = np.array(y_rest[idx].tolist() + y_.tolist())

  mod_iso_ = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=200), n_estimators=number_of_subsets)
  model_iso_ = CalibratedClassifierCV(base_estimator=mod_iso_, cv=3, method='isotonic')
  model_iso_.fit(x_, y_)

  joblib.dump(model_iso_, path + f'/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty_randomX4_{i}.joblib')

  x_rest = np.delete(x_rest, idx, axis=0)
  y_rest = np.delete(y_rest, idx, axis=0)

1/10...
2/10...
3/10...
4/10...
5/10...
6/10...
7/10...
8/10...
9/10...
10/10...


In [ ]:
# Ανάκληση των αποθηκευμένων μοντέλων με τα σταδιακά αυξανόμενα δεδομένα και εκτύπωση στατιστικών

for i in range(10):
  model_iso_ = joblib.load(path + f'/iso_bal_bag_rf_200_gini_200_active_learning_uncertainty_randomX4_{i}.joblib')

  y_preds_ = model_iso_.predict_proba(x_te.to_numpy())

  y_preds_ = y_preds_.argmax(axis=1)

  print(f'Μετά από πρόσθεση {i+1}x5000 παραδειγμάτων...')
  print(confusion_matrix(y_te, y_preds_).T)
  print()
  print(classification_report(y_te, y_preds_))
  print(50*'-')

Μετά από πρόσθεση 1x5000 παραδειγμάτων...
[[85280    35]
 [   16   112]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.88      0.76      0.81       147

    accuracy                           1.00     85443
   macro avg       0.94      0.88      0.91     85443
weighted avg       1.00      1.00      1.00     85443

--------------------------------------------------
Μετά από πρόσθεση 2x5000 παραδειγμάτων...
[[85281    35]
 [   15   112]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.88      0.76      0.82       147

    accuracy                           1.00     85443
   macro avg       0.94      0.88      0.91     85443
weighted avg       1.00      1.00      1.00     85443

--------------------------------------------------
Μετά από πρόσθεση 3x5000 παραδειγμάτων...
[[85280    34]
 [   16   113]]

              precisio

In [ ]:
# Εκπαίδευση μοντέλου Logistic Regression στο αρχικό σύνολο δεδομένων και εκτύπωση στατιστικών.

y0 = y_tr[y_tr==0]
x0 = x_tr[y_tr==0]

y1 = y_tr[y_tr==1]
x1 = x_tr[y_tr==1]

ln = len(y1)

model = LogisticRegression(max_iter=1000)
model.fit(x_tr, y_tr)

y_preds = model.predict_proba(x_te.to_numpy())

y_preds = y_preds.argmax(axis=1)

print(confusion_matrix(y_te, y_preds).T)
print()
print(classification_report(y_te, y_preds))

[[85283    60]
 [   13    87]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.87      0.59      0.70       147

    accuracy                           1.00     85443
   macro avg       0.93      0.80      0.85     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
a = dt.columns.to_list()
a.remove('Class')
print(model.coef_)
l = list(zip(model.coef_[0], a))
l.sort(key=lambda x: abs(x[0]))
l

[[ 0.38803134 -0.16525024  0.02420084  0.67970541 -0.12276127 -0.23048823
   0.04879186 -0.1768899  -0.54344013 -1.01497502 -0.02362276  0.84775797
  -0.58645652 -0.98540807  0.09713806  0.32949722 -0.47109196 -0.21616728
   0.25943245 -0.29556051  0.41833702  0.81312567 -0.37925127  0.09884945
  -0.29995207  0.01845517 -1.08430872 -1.02804958  0.08181281 -0.3739041
  -0.06345761  0.15559366]]


[(0.018455174674810223, 'V26'),
 (-0.02362275572394452, 'V11'),
 (0.024200842648078702, 'V3'),
 (0.048791857058640184, 'V7'),
 (-0.06345760541771688, 'Actual_Time_cos'),
 (0.08181280998788504, 'Amount'),
 (0.09713805944577461, 'V15'),
 (0.09884945300960875, 'V24'),
 (-0.12276126969458019, 'V5'),
 (0.15559366296107438, 'Actual_Time_sin'),
 (-0.1652502371262837, 'V2'),
 (-0.17688989764755703, 'V8'),
 (-0.21616728384285283, 'V18'),
 (-0.23048823354760276, 'V6'),
 (0.2594324497903161, 'V19'),
 (-0.2955605116550289, 'V20'),
 (-0.29995207300643295, 'V25'),
 (0.32949722448275554, 'V16'),
 (-0.37390409874548486, 'Density'),
 (-0.3792512669917476, 'V23'),
 (0.38803133536536977, 'V1'),
 (0.41833701766553694, 'V21'),
 (-0.47109195981051616, 'V17'),
 (-0.5434401305783655, 'V9'),
 (-0.5864565236582673, 'V13'),
 (0.679705406140543, 'V4'),
 (0.8131256731629494, 'V22'),
 (0.8477579697890858, 'V12'),
 (-0.9854080735984864, 'V14'),
 (-1.014975024367036, 'V10'),
 (-1.0280495767372415, 'V28'),
 (-1.084308

In [ ]:
# Πρόβλεψη των πιθανοτήτων για όλα τα μη ετικετοποιημένα δεδομένα και επιλογή των 50000 πιο αβέβαιων

cost = 5000
cost_per_example = 0.1

preds = model.predict_proba(x_UN.to_numpy())

idx = np.argsort(np.abs(preds[:,0].flatten() - 0.5))[:int(cost/cost_per_example)]

In [ ]:
# Εκπαίδευση του μοντέλου με τα νέα επαυξημένα δεδομένα και εκτύπωση στατιστικών.

x_ = np.vstack([x_UN.to_numpy()[idx], x_tr])
y_ = y_UN.to_numpy()[idx].tolist() + y_tr.to_list()

model_ = LogisticRegression(max_iter=1000)
model_.fit(x_, y_)

y_preds_ = model_.predict_proba(x_te.to_numpy())

y_preds_ = y_preds_.argmax(axis=1)

print(confusion_matrix(y_te, y_preds_).T)
print()
print(classification_report(y_te, y_preds_))


[[85286    49]
 [   10    98]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.91      0.67      0.77       147

    accuracy                           1.00     85443
   macro avg       0.95      0.83      0.88     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
# Σταδιακή επέκταση των δεδομένων εκπαίδευσης σε 10 βήματα (ανά 5000 δείγματα) και εκτύπωση στατιστικών.

cost = 5000
cost_per_example = 0.1


sps = np.linspace(0, int(cost/cost_per_example), 11)
x_rest = x_UN.to_numpy()
y_rest = y_UN.to_numpy()
x_ = x_tr.to_numpy()
y_ = y_tr.to_numpy()

for i in range(10):
  print(f'{i+1}/10...')
  sp = int(sps[i+1] - sps[i])

  preds = model.predict_proba(x_rest)

  idx = np.argsort(np.abs(preds[:,0].flatten() - 0.5))[:sp]

  x_ = np.vstack([x_rest[idx], x_])
  y_ = np.array(y_rest[idx].tolist() + y_.tolist())

  model_ = LogisticRegression(max_iter=1000)
  model_.fit(x_, y_)

  y_preds_ = model_.predict_proba(x_te.to_numpy())

  y_preds_ = y_preds_.argmax(axis=1)

  print(f'Μετά από πρόσθεση {i+1}x5000 παραδειγμάτων...')
  print(confusion_matrix(y_te, y_preds_).T)
  print()
  print(classification_report(y_te, y_preds_))
  print(50*'-')

  x_rest = np.delete(x_rest, idx, axis=0)
  y_rest = np.delete(y_rest, idx, axis=0)

1/10...
Μετά από πρόσθεση 1x5000 παραδειγμάτων...
[[85284    48]
 [   12    99]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.89      0.67      0.77       147

    accuracy                           1.00     85443
   macro avg       0.95      0.84      0.88     85443
weighted avg       1.00      1.00      1.00     85443

--------------------------------------------------
2/10...
Μετά από πρόσθεση 2x5000 παραδειγμάτων...
[[85285    48]
 [   11    99]]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85296
           1       0.90      0.67      0.77       147

    accuracy                           1.00     85443
   macro avg       0.95      0.84      0.89     85443
weighted avg       1.00      1.00      1.00     85443

--------------------------------------------------
3/10...
Μετά από πρόσθεση 3x5000 παραδειγμάτων...
[[85285    48]
 [   11    99]]